# Convolutional kernels - blurring, thresholds, edge detection

In [ ]:
# We need to include the home directory in our path, so we can read in our own module.
import os
import sys
sys.path.append(os.path.join(".."))
import cv2
import numpy as np
from utils.imutils import jimshow
from utils.imutils import jimshow_channel
import matplotlib.pyplot as plt

### Load image

Let's start by loading our dino buddy.

In [ ]:
filename = os.path.join("..", "data", "img", "trex.png")

In [ ]:
image = cv2.imread(filename)

In [ ]:
jimshow(image, "Dino")

## Blurring and convolutions

```OpenCV``` has built in functions for performing blurring.

All we need to do is choose which blurring method we want, and some parameters which define the blur.

__Averaging__

In [ ]:
blurred = np.hstack([
    cv2.blur(image, (3, 3)),
    cv2.blur(image, (5, 5)),
    cv2.blur(image, (7, 7))]
    )
jimshow(blurred)

__Gaussian__

Third parameter relates to variance around the mean for a Gaussian distribution. For our purposes, we can just set that to ```0```.


In [ ]:
# gaussian blur
blurred = np.hstack([
    cv2.GaussianBlur(image, (3, 3), 0),
    cv2.GaussianBlur(image, (5, 5), 0),
    cv2.GaussianBlur(image, (7, 7), 0)]
    )
jimshow(blurred)

__Median__

In [ ]:
#does not "invent" pixel values, kernel removes details and noise
blurred = np.hstack([
    cv2.medianBlur(image, 3),
    cv2.medianBlur(image, 5),
    cv2.medianBlur(image, 7)]
    )
jimshow(blurred)

## Thresholding

As with blurring, ```OpenCV``` has built in functions for allowing us to perform thresholding. However, we need to perform another couple of steps before we get there.

__Simple thresholding__

We'll start by reloading the image and converting to greyscale with ```cv2.cvtColor()``` like we saw a couple of weeks back.

In [ ]:
image = cv2.imread(filename)

In [ ]:
# convert to greyscale
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

When using simple thresholding, we want to set a single specific value above which everything is set to white and below which everything is set to black.

How do we determine the value?

In [ ]:
# plot histogram
plt.hist(image.flatten(),256,[0,256])
plt.show()

__Blur the image__

To get the best results for our edge detection, we first need to blur our image to smooth out the noise.

In [ ]:
# remove high frequency edges with a blur kernel
blurred = cv2.GaussianBlur(image, (5, 5), 0)
jimshow_channel(image)

Once we've blurred the image, we then use the ```cv2.threshold()``` function to perform simple thresholding using our predetermined binary threshold.

In [ ]:
# threshold 
(T, thres) = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
jimshow_channel(thres)

This gives us a black sillhouette on a white background. However, when working with a mask, we want the opposite because we're using ```bitwise_AND```.

In [ ]:
(T, thresInv) = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)
jimshow_channel(thresInv)

We can then display the image using the mask generated by our simple threshold.

In [ ]:
jimshow_channel(cv2.bitwise_and(image, image, mask = thresInv))

__Adaptive__

The exact same steps are required when performing adaptive thresholding. We load the image, make it greyscale, and use blurring. 

The only difference is that, this time, we use the ```cv2.adaptiveThreshold()``` function.

In [ ]:
image = cv2.imread(filename)

In [ ]:
# preprocessing
image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(image_grey, (5,5), 0)

__Adaptive mean threshold__

The ```cv2.adaptiveTreshold()``` function requires slightly different combinations of arguments, outlined below.

In [ ]:
am_thresh = cv2.adaptiveThreshold(blurred, 
    255,                            # set values above threshold to white
    cv2.ADAPTIVE_THRESH_MEAN_C,     # method for computing threshold (mean of neighborhood) 
    cv2.THRESH_BINARY_INV,          # method for thresholding 
    5,                              # neighborhood pixel size
    10                              # parameter C subtracted from the mean to fine-tune thresholding
    )
jimshow_channel(am_thresh)

Once we've found the edges, we can then use it as a mask, just as with simple thresholding.

In [ ]:
jimshow_channel(cv2.bitwise_and(image_grey, image_grey, mask = am_thresh))

## Edge Detection

Edge detection follows a similar set of steps again. We always begin by loading the image and making it greyscale.

In [ ]:
newfile = os.path.join("..", "data", "img", "coins.png")
image = cv2.imread(newfile)
grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## Sobel operators

We can use the ```cv2.Sobel()``` function to define horiztonal and vertical sobel kernels.

In [ ]:
# Sobel operatator
sobelX = cv2.Sobel(grey, cv2.CV_64F, 1, 0)  # horizontal kernel
sobelY = cv2.Sobel(grey, cv2.CV_64F, 0, 1)  # vertical kernel

Notice that here we're using 64-bit floating point numbers, while all of our pixel values are actually unsigned 8-bit integers. The reason for this is that ```float64``` values can show much more finegrained variation, which allow us to detect edges much more accurately.

In order to work with the edges for the rest of our workflow, we need to convert back to unsiged integers, which we can do in the following way:

In [ ]:
sobelX = np.uint8(np.absolute(sobelX))
sobelY = np.uint8(np.absolute(sobelY))

We can then see how these look:

In [ ]:
blurred = np.hstack([sobelX, sobelY])
jimshow_channel(blurred, "Horizontal vs vertical Sobel")

## Combined Sobel

In order to combine the gradient images in both the x and y direction, we can apply a bitwise OR. 

Remember, an OR operation is true when either pixel is greater than zero. 

Therefore, a given pixel will be ```True``` if either a horizontal or vertical edge is present.

In [ ]:
sobelCombined = cv2.bitwise_or(sobelX, sobelY)

In [ ]:
jimshow_channel(sobelCombined)

## Laplacian

Laplacian operator returns a single gradient, no need to perform two passes.

In [ ]:
#laplacian to compute gradient magnitude
lap = cv2.Laplacian(grey, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))

In [ ]:
jimshow_channel(lap)